In [142]:
amino_dict={
        'ALA':'A',
        'ARG':'R',
        'ASN':'N',
        'ASP':'D',
        'CYS':'C',
        'GLU':'E',
        'GLN':'Q',
        'GLY':'G',
        'HIS':'H',
        'ILE':'I',
        'LEU':'L',
        'LYS':'K',
        'MET':'M',
        'PHE':'F',
        'PRO':'P',
        'SER':'S',
        'THR':'T',
        'TRP':'W',
        'TYR':'Y',
        'VAL':'V'
        
    }

def amino_name(a):
    amino_dict={
        'ALA':'A',
        'ARG':'R',
        'ASN':'N',
        'ASP':'D',
        'CYS':'C',
        'GLU':'E',
        'GLN':'Q',
        'GLY':'G',
        'HIS':'H',
        'ILE':'I',
        'LEU':'L',
        'LYS':'K',
        'MET':'M',
        'PHE':'F',
        'PRO':'P',
        'SER':'S',
        'THR':'T',
        'TRP':'W',
        'TYR':'Y',
        'VAL':'V'
        
    }
    return amino_dict[a]

In [143]:

import os

data = open("/Users/kistintern6/set2.antigen/2fd6_ag.pdb") #pdb파일을 열어서 data에 저장
def parsing_pdb(pdb_file):
    with open(pdb_file, 'r') as file:
        xyz={}
        for line in file:
            if line.startswith("ATOM"):
                atom_info={
                    #'Serial_Num':line[6:11],
                    #'Atom_Name':line[12:16].strip(),
                    #'Alt_location':line[16:17],
                    'Residue_Name':line[17:20].strip(),
                    'Residue_num':int(line[22:26].strip()),
                    #'xyz':line[30:54].strip()
                }

                if atom_info['Residue_num'] not in xyz:
                    xyz[atom_info['Residue_num']]=atom_info['Residue_Name']
                
                
    return xyz





In [144]:
#day1. atom 좌표 저장
import numpy as np

#insertion dictionary
data = open("/Users/kistintern6/set2.antigen/2fd6_ag.pdb") #pdb파일을 열어서 data에 저장
atom_dict = {} #3중 딕셔너리임. chain에 해당하는 residue에 있는 atom에 대한 좌표값
count=0
error_dict={}
error_list=[]
new_atom={}
seq_dict={}
for line in data:#data 한 줄씩 가져오기
            
    if line[:4] == 'ATOM': #ATOM으로 시작하는 line 찾기
        
        chain = line[21] #chain identifer
        if chain not in atom_dict:
            count=0
            error_list.clear()
            atom_dict[chain] = {}
            
        if chain not in new_atom:
            new_atom[chain]={}
            
        if chain not in error_dict:
            error_dict[chain]={}
            
        if chain not in seq_dict:
            seq_dict[chain]={}
        
        resn=line[17:20].strip()
        atom_name = line[12:16] #원자 이름
        atom_name_split = ('').join(atom_name.split()) #원자이름 공백 제거
        resi = line[22:27].strip()
        real_index=line[22:27].strip()
                   
        
        if real_index not in new_atom[chain]:
            new_atom[chain][real_index]=0
            
        try:
            resi=int(resi)+count
        except:
            if int(resi[:-1])==1:
                if resi not in error_list:
                    error_list.append(resi)
                    count+=1
                resi=int(resi[:-1])
            else:
                if resi not in error_list:
                    error_list.append(resi)
                    count+=1
                resi=int(resi[:-1])+count
  
            
            
        error_dict[chain]=error_list
        
        if resi not in atom_dict[chain]:
            #atom_dict[chain]에 num 있는지 확인
            atom_dict[chain][resi] = {}
        atom_dict[chain][resi][atom_name_split] = np.array([float(line[30:38]), float(line[38:46]), float(line[46:54])])
        
        if resi not in seq_dict[chain]:
            seq_dict[chain][resi]=amino_name(resn)
            
           
            #print(resi)
        
       
        
#print(error_list)  
#print(seq_dict)     
print()
print(atom_dict)
#print(atom_dict['B'])
data.close()







                



{'U': {1: {'N': array([ 10.491, -48.293,  66.042]), 'CA': array([ 11.34 , -49.053,  67.   ]), 'C': array([ 12.713, -49.397,  66.404]), 'O': array([ 13.15 , -50.55 ,  66.462]), 'CB': array([ 10.608, -50.336,  67.432]), 'OG': array([ 10.213, -51.125,  66.316])}, 2: {'N': array([ 13.391, -48.394,  65.843]), 'CA': array([ 14.713, -48.587,  65.242]), 'C': array([ 15.794, -48.269,  66.266]), 'O': array([ 15.63 , -47.375,  67.085]), 'CB': array([ 14.893, -47.649,  64.039]), 'CG': array([ 15.888, -47.939,  62.902]), 'CD1': array([ 16.615, -46.661,  62.524]), 'CD2': array([ 16.881, -48.993,  63.311])}, 3: {'N': array([ 16.898, -49.007,  66.222]), 'CA': array([ 18.022, -48.765,  67.126]), 'C': array([ 19.284, -48.679,  66.285]), 'O': array([ 19.43 , -49.404,  65.296]), 'CB': array([ 18.19 , -49.9  ,  68.132]), 'CG': array([ 17.024, -50.131,  69.059]), 'CD': array([ 17.448, -51.091,  70.152]), 'NE': array([ 17.928, -52.36 ,  69.607]), 'CZ': array([ 18.756, -53.186,  70.242]), 'NH1': array([ 19.2

In [145]:
#day2-3. phi/psi 계산

import numpy as np
import math



def return_angle(A, B, C, D):
    AB = B - A
    AC = C - A
    BC = C - B
    BD = D - B
    v1 = np.cross(AB, AC) #np.cross : 벡터의 외적
    v2 = np.cross(BC, BD)
    v1v2 = np.dot(v1, v2) #np.dot : 벡터 점의 곱
    len_v1 = math.sqrt(np.sum(v1*v1)) #np.sqrt : 제곱근, np.sum : 합
    len_v2 = math.sqrt(np.sum(v2*v2))
    radi = math.acos(v1v2 / (len_v1*len_v2)) #math.acos : radian구하기
    x = math.degrees(radi) #math.degrees : 라디안을 각도로 변환
    k = np.dot(np.cross(v1,v2), BC)
    sign = 1
    if k < 0:
        sign = -1
    return (sign*x)

def phi_cal_angle(atom_dict, num): #좌표찾고 phi 계산
    if 'C' in atom_dict[num - 1] and 'N' in atom_dict[num] and 'CA' in atom_dict[num] and 'C' in atom_dict[num]:
        phi = return_angle(atom_dict[num - 1]['C'], atom_dict[num]['N'], atom_dict[num]['CA'], atom_dict[num]['C'])
    else:
        phi = None
    return (phi)

def psi_cal_angle(atom_dict, num): #각각의 좌표 찾아서 psi계산
    if 'N' in atom_dict[num] and 'CA' in atom_dict[num] and 'C' in atom_dict[num] and 'N' in atom_dict[num + 1]:
        psi = return_angle(atom_dict[num]['N'], atom_dict[num]['CA'], atom_dict[num]['C'], atom_dict[num + 1]['N'])
    else:
        psi = None
    return(psi)

phi_dict = {}
psi_dict = {}
for ch in atom_dict:
    if ch not in phi_dict:
        phi_dict[ch] = {}
    if ch not in psi_dict:
        psi_dict[ch] = {}
    for num in atom_dict[ch]:
        if num - 1 in atom_dict[ch]:
            phi = phi_cal_angle(atom_dict[ch], num)
            if phi is not None:
                phi_dict[ch][num] = phi
        if num + 1 in atom_dict[ch]:
            psi = psi_cal_angle(atom_dict[ch], num)
            if psi is not None:
                psi_dict[ch][num] = psi_cal_angle(atom_dict[ch], num)


print("phi---------------------------------------------")
print(phi_dict)
print("psi---------------------------------------------")
print(psi_dict)
print("------------------------------------------------")

phi---------------------------------------------
{'U': {2: -93.99727964864819, 3: -129.62560724811178, 4: -139.57177978620288, 5: -75.22326303711642, 6: -119.12866884493272, 7: -122.54160745680022, 8: -91.2076163904102, 9: -72.97834808905088, 10: -73.0583533635601, 11: 104.75969844000873, 12: -69.23697542532302, 13: -112.76655541697075, 14: -168.43097148191217, 15: -81.15599849334119, 16: -118.71170591402719, 17: -90.00184889179575, 18: -55.149611753501596, 19: -86.26475984638557, 20: -46.059048886975944, 21: 106.34688881743597, 22: -96.62471711060658, 23: -105.50005037750769, 24: -143.01518691406864, 25: -107.21620183888112, 26: -136.55420336348436, 27: -124.88489613801484, 28: -100.4809667314363, 29: -120.67924227213427, 30: -124.96016318281876, 31: -128.4609643020308, 32: -120.05829308522213, 33: -105.54991561189364, 34: -163.42488833680397, 39: -127.22052751757879, 40: -120.65986718965895, 41: -107.69523755745377, 42: -130.6452626909795, 43: -143.514508173618, 44: -145.023119236469

In [146]:
NOdict={}
for ch in atom_dict:
    if ch not in NOdict:
        NOdict[ch]={}
        for i in atom_dict[ch]:
            if i not in NOdict[ch]:
                NOdict[ch][i]={
                    'N':{},
                    'O':{}
                }
                
            NOdict[ch][i]['O']=atom_dict[ch][i].get('O')
            NOdict[ch][i]['N']=atom_dict[ch][i].get('N')                

print(NOdict)

{'U': {1: {'N': array([ 10.491, -48.293,  66.042]), 'O': array([ 13.15 , -50.55 ,  66.462])}, 2: {'N': array([ 13.391, -48.394,  65.843]), 'O': array([ 15.63 , -47.375,  67.085])}, 3: {'N': array([ 16.898, -49.007,  66.222]), 'O': array([ 19.43 , -49.404,  65.296])}, 4: {'N': array([ 20.196, -47.797,  66.677]), 'O': array([ 22.483, -47.042,  68.011])}, 5: {'N': array([ 23.815, -47.743,  66.343]), 'O': array([ 25.14 , -45.389,  66.083])}, 6: {'N': array([ 25.605, -45.504,  68.271]), 'O': array([ 27.634, -44.267,  70.017])}, 7: {'N': array([ 28.243, -43.608,  67.958]), 'O': array([ 29.867, -41.469,  67.086])}, 8: {'N': array([ 31.103, -41.664,  68.949]), 'O': array([ 33.277, -41.863,  67.722])}, 9: {'N': array([ 33.808, -39.683,  67.702]), 'O': array([ 37.004, -41.225,  67.324])}, 10: {'N': array([ 35.95 , -40.453,  69.158]), 'O': array([ 37.687, -43.214,  70.813])}, 11: {'N': array([ 35.755, -43.149,  69.67 ]), 'O': array([ 34.147, -46.186,  70.769])}, 12: {'N': array([ 34.077, -44.081,

In [147]:
def cal_len(A, B): #길이 계산
	P = A - B
	len = np.sqrt(np.sum(P*P))
	return (len)

Hdict = {}
for ch in atom_dict: #residue_dict에 [O,N] residue 저장
	if ch not in Hdict:
		Hdict[ch] = {}
		residue = []
	for num in atom_dict[ch]:
		for i in atom_dict[ch]:
			if num != i and 'O' in atom_dict[ch][num] and 'N' in atom_dict[ch][i]:
				len = cal_len(atom_dict[ch][num]['O'], atom_dict[ch][i]['N'])
				if 2.5 < len < 3.5 :
					residue.append([num, i])
	Hdict[ch] = residue
 
print(Hdict)

{'U': [[2, 18], [3, 76], [4, 6], [4, 16], [5, 45], [6, 14], [7, 44], [8, 10], [8, 11], [9, 11], [11, 13], [12, 8], [14, 6], [16, 4], [16, 18], [19, 21], [19, 22], [21, 23], [24, 47], [25, 72], [26, 45], [27, 70], [28, 43], [29, 68], [30, 41], [31, 66], [32, 39], [33, 64], [39, 32], [41, 30], [43, 28], [44, 7], [45, 26], [47, 24], [48, 50], [48, 51], [49, 51], [52, 54], [54, 69], [55, 150], [56, 67], [57, 148], [58, 65], [63, 60], [64, 33], [65, 58], [66, 31], [67, 56], [68, 29], [69, 54], [70, 27], [72, 25], [74, 76], [74, 77], [75, 77], [76, 78], [76, 79], [94, 116], [95, 176], [96, 114], [97, 147], [98, 112], [99, 146], [100, 105], [100, 106], [101, 103], [101, 104], [103, 105], [104, 108], [105, 109], [105, 110], [106, 108], [106, 109], [108, 110], [109, 111], [112, 98], [114, 96], [118, 120], [118, 121], [122, 149], [123, 172], [124, 147], [125, 170], [126, 145], [127, 168], [129, 166], [141, 143], [142, 144], [143, 128], [144, 101], [145, 126], [146, 99], [147, 124], [148, 57], [1

In [148]:
#alpha-helix
def H_func(ss_dict,Hdict,phi_dict,psi_dict):

    for L in Hdict:
        if L[0]+4==L[1]:
            for i in range(L[0],L[1]+1):
                if ((i not in phi_dict) and (-90<psi_dict[i]<30)):
                    ss_dict[i]="H"
                elif ((i not in psi_dict) and (-150<phi_dict[i]<-30)):
                    ss_dict[i]="H"
                elif ((-150<phi_dict[i]<-30) and (-90<psi_dict[i]<30)):
                    ss_dict[i]="H"
    return ss_dict

In [149]:
#beta-helix
#angle
def E_angle(ss_dict,atom_dict,phi_dict,psi_dict):
    
    for i in atom_dict:
        if (i in phi_dict and phi_dict[i]> -20) or (i in psi_dict and psi_dict[i] < 45):
            if not ss_dict[i]:
                ss_dict[i]="C"
    return (ss_dict)
                
def E_func(ss_dict,Hdict):
    for L in Hdict:
        i=L[0]
        j=L[1]    
        if [i-2, j-2] in Hdict:
            for x in range(i-2,i+1):
                if (ss_dict[x]=="H") or (ss_dict[x]=="C"):
                    pass
                elif x in ss_dict:
                    ss_dict[x]="E"
                
                
        elif [i+2, j-2] in Hdict:
            for x in range(i,i+3):
                if (ss_dict[x]=="H") or (ss_dict[x]=="C"):
                    pass
                elif x in ss_dict:
                    ss_dict[x]="E"
    
    for L in Hdict:
        i=L[1]
        j=L[0]    
        if [j-2, i-2] in Hdict:
            for x in range(i-2,i+1):
                if (ss_dict[x]=="H") or (ss_dict[x]=="C"):
                    pass
                elif x in ss_dict:
                    ss_dict[x]="E"
                
                
        elif [j+2, i-2] in Hdict:
            for x in range(i-2,i+1):
                if (ss_dict[x]=="H") or (ss_dict[x]=="C"):
                    pass
                elif x in ss_dict:
                    ss_dict[x]="E"
                    
    return ss_dict

'''
def E_cal(ss_dict, residue):
    for L in residue:
        i=L[0]
        j=L[1]
        if i-2>0 and j-2>0:
            if [i-2,j-2] in residue:
                ss_dict[i]="E"
        elif j-2>0:
            if [i+2,j-2] in residue:
                ss_dict[i]="E"
    return ss_dict
    '''
        

'\ndef E_cal(ss_dict, residue):\n    for L in residue:\n        i=L[0]\n        j=L[1]\n        if i-2>0 and j-2>0:\n            if [i-2,j-2] in residue:\n                ss_dict[i]="E"\n        elif j-2>0:\n            if [i+2,j-2] in residue:\n                ss_dict[i]="E"\n    return ss_dict\n    '

In [150]:
#coil
def C_func(ss_dict, atom_dict):
    for i in atom_dict:
        if (ss_dict[i]=="H") or (ss_dict[i]=="E"):
            pass
        else:
            ss_dict[i]="C"
    
    return ss_dict

def check_C(ss_dict, atom_dict):
    for i in atom_dict:
         if ss_dict[i]=="E":
             if (i-1 in ss_dict and ss_dict[i-1]!="E") or (i+1 in ss_dict and ss_dict[i+1]!="E"):
                 if (i+3 in ss_dict and ss_dict[i+2]!="E") and (i-3 in ss_dict and ss_dict[i-2]!="E"):
                     ss_dict[i]="C"
    return ss_dict

In [151]:
ss_dict={}
for ch in atom_dict:
    
    ss_dict[ch]={}
    
    for i in atom_dict[ch]:
        ss_dict[ch][i]={}

    b=H_func(ss_dict[ch],Hdict[ch],phi_dict[ch],psi_dict[ch])
    ss_dict[ch]=b
    c=E_angle(ss_dict[ch],atom_dict[ch],phi_dict[ch],psi_dict[ch])
    ss_dict[ch]=c
    d=E_func(ss_dict[ch],Hdict[ch])
    ss_dict[ch]=d
    a=C_func(ss_dict[ch],atom_dict[ch])
    ss_dict[ch]=a
    
print(ss_dict)

#c=E_angle(ss_dict,atom_dict,phi_dict,psi_dict)
#ss_dict[ch]=c
#print(ss_dict)





{'U': {1: 'C', 2: 'E', 3: 'E', 4: 'E', 5: 'E', 6: 'E', 7: 'E', 8: 'E', 9: 'C', 10: 'C', 11: 'C', 12: 'E', 13: 'E', 14: 'E', 15: 'E', 16: 'E', 17: 'E', 18: 'E', 19: 'E', 20: 'E', 21: 'C', 22: 'E', 23: 'C', 24: 'E', 25: 'E', 26: 'E', 27: 'E', 28: 'E', 29: 'E', 30: 'E', 31: 'E', 32: 'E', 33: 'E', 34: 'C', 38: 'C', 39: 'E', 40: 'E', 41: 'E', 42: 'E', 43: 'E', 44: 'E', 45: 'C', 46: 'E', 47: 'E', 48: 'C', 49: 'C', 50: 'C', 51: 'C', 52: 'C', 53: 'C', 54: 'E', 55: 'E', 56: 'E', 57: 'E', 58: 'E', 59: 'E', 60: 'E', 61: 'C', 62: 'C', 63: 'E', 64: 'E', 65: 'E', 66: 'E', 67: 'E', 68: 'E', 69: 'E', 70: 'E', 71: 'E', 72: 'E', 73: 'C', 74: 'E', 75: 'E', 76: 'C', 77: 'C', 78: 'C', 79: 'C', 80: 'C', 81: 'C', 93: 'C', 94: 'E', 95: 'E', 96: 'E', 97: 'E', 98: 'E', 99: 'E', 100: 'E', 101: 'C', 102: 'C', 103: 'C', 104: 'C', 105: 'H', 106: 'H', 107: 'H', 108: 'H', 109: 'C', 110: 'E', 111: 'C', 112: 'E', 113: 'E', 114: 'E', 115: 'E', 116: 'E', 117: 'C', 118: 'C', 119: 'C', 120: 'C', 121: 'C', 122: 'E', 123: 'E

In [152]:
#왜 빵꾸나는지 파악=>해결!
#시퀀스 마무리
'''
ss_list=[]
count_zero=0
for ch in ss_dict:
    count_zero=0
    for i in ss_dict[ch]:
        ss_list.append(ss_dict[ch][i])
        count_zero+=1

    print(count_zero)
'''


for ch in new_atom:
    ss_list_index=0
    count_zero=0
    seq_list=[]
    ss_list=[]
    for i in ss_dict[ch]:
        ss_list.append(ss_dict[ch][i])
        count_zero+=1
    #print(ss_list)
    for k in seq_dict[ch]:
        seq_list.append(seq_dict[ch][k])
    #print(seq_list)
    #print()    
    for j in new_atom[ch]:
        if ss_list_index<count_zero:
    
            new_atom[ch][j]=[ss_list[ss_list_index]]
            new_atom[ch][j].append(seq_list[ss_list_index])
            ss_list_index+=1
        else:
            break

            
print(new_atom)

{'U': {'1A': ['C', 'S'], '1': ['E', 'L'], '2': ['E', 'R'], '3': ['E', 'C'], '4': ['E', 'M'], '5': ['E', 'Q'], '6': ['E', 'C'], '7': ['E', 'K'], '8': ['C', 'T'], '9': ['C', 'N'], '10': ['C', 'G'], '11': ['E', 'D'], '12': ['E', 'C'], '13': ['E', 'R'], '14': ['E', 'V'], '15': ['E', 'E'], '16': ['E', 'E'], '17': ['E', 'C'], '18': ['E', 'A'], '19': ['E', 'L'], '20': ['C', 'G'], '21': ['E', 'Q'], '22': ['C', 'D'], '23': ['E', 'L'], '24': ['E', 'C'], '25': ['E', 'R'], '26': ['E', 'T'], '27': ['E', 'T'], '28': ['E', 'I'], '29': ['E', 'V'], '30': ['E', 'R'], '31': ['E', 'L'], '32': ['E', 'W'], '33': ['C', 'E'], '37': ['C', 'E'], '38': ['E', 'L'], '39': ['E', 'E'], '40': ['E', 'L'], '41': ['E', 'V'], '42': ['E', 'E'], '43': ['E', 'K'], '44': ['C', 'S'], '45': ['E', 'C'], '46': ['E', 'T'], '47': ['C', 'H'], '48': ['C', 'S'], '49': ['C', 'E'], '50': ['C', 'K'], '51': ['C', 'T'], '52': ['C', 'N'], '53': ['E', 'R'], '54': ['E', 'T'], '55': ['E', 'L'], '56': ['E', 'S'], '57': ['E', 'Y'], '58': ['E', 

In [153]:
#Helix 검증
for ch in atom_dict:
    a=[k for k,v in ss_dict[ch].items() if v=="H"]
    print(a)

[105, 106, 107, 108, 208, 209, 210, 254, 255, 256, 257]


In [154]:
#Sheet 검증
count=0
for ch in atom_dict:
    a=[k for k,v in ss_dict[ch].items() if v=="E"]
    count+=1
    print(a)

    print()
    

[2, 3, 4, 5, 6, 7, 8, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 39, 40, 41, 42, 43, 44, 46, 47, 54, 55, 56, 57, 58, 59, 60, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 94, 95, 96, 97, 98, 99, 100, 110, 112, 113, 114, 115, 116, 122, 123, 124, 125, 126, 127, 128, 129, 144, 145, 146, 148, 150, 157, 158, 159, 160, 161, 165, 166, 167, 168, 169, 170, 171, 172, 174, 175, 179, 193, 194, 195, 196, 197, 199, 212, 213, 214, 215, 216, 217, 220, 222, 223, 224, 225, 226, 227, 229, 235, 236, 237, 238, 239, 240, 242, 243, 244, 249, 263, 264, 265, 266, 267, 269, 270, 274]



In [155]:
#Coil 검증
for ch in atom_dict:
    a=[k for k,v in ss_dict[ch].items() if v=="C"]
    print(a)
    print()

[1, 9, 10, 11, 21, 23, 34, 38, 45, 48, 49, 50, 51, 52, 53, 61, 62, 73, 76, 77, 78, 79, 80, 81, 93, 101, 102, 103, 104, 109, 111, 117, 118, 119, 120, 121, 130, 141, 142, 143, 147, 149, 151, 152, 153, 154, 155, 156, 162, 163, 164, 173, 176, 177, 178, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 198, 200, 201, 202, 203, 204, 205, 206, 207, 211, 218, 219, 221, 228, 230, 231, 232, 233, 234, 241, 245, 246, 247, 248, 253, 258, 259, 260, 261, 262, 268, 271, 272, 273, 275, 276]



In [156]:
amino_dict={
        'ALA':'A',
        'ARG':'R',
        'ASN':'N',
        'ASP':'D',
        'CYS':'C',
        'GLU':'E',
        'GLN':'Q',
        'GLY':'G',
        'HIS':'H',
        'ILE':'I',
        'LEU':'L',
        'LYS':'K',
        'MET':'M',
        'PHE':'F',
        'PRO':'P',
        'SER':'S',
        'THR':'T',
        'TRP':'W',
        'TYR':'Y',
        'VAL':'V'
        
    }

def amino_name(a):
    amino_dict={
        'ALA':'A',
        'ARG':'R',
        'ASN':'N',
        'ASP':'D',
        'CYS':'C',
        'GLU':'E',
        'GLN':'Q',
        'GLY':'G',
        'HIS':'H',
        'ILE':'I',
        'LEU':'L',
        'LYS':'K',
        'MET':'M',
        'PHE':'F',
        'PRO':'P',
        'SER':'S',
        'THR':'T',
        'TRP':'W',
        'TYR':'Y',
        'VAL':'V'
        
    }
    return amino_dict[a]

In [157]:
from collections import OrderedDict

for ch in new_atom:
    new_string=""
    ordered_dict=OrderedDict(new_atom[ch])
    for key,value in ordered_dict.items():
        new_string+=value[1]
    
    print(ch)
    print(new_string)
    print()

U
SLRCMQCKTNGDCRVEECALGQDLCRTTIVRLWEELELVEKSCTHSEKTNRTLSYRTGLKITSLTEVVCGLDLCNQGNYLECISCGSSDMSCERGRHQSLQCRSPEEQCLDVVTHWDDRHLRGCGYLPGCPGSNGFHNNDTFHFLKCCNTTKCNEGPILELENLPQNGRQCYSCKGNSTHGCSSEETFLIDCRGPMNQCLVATGTHEPKNQSYMVRGCATASMCQLGDAFSMNHIDVSCCTKSGCNHPD

